In [ ]:
#conda activate scan_snpeff

In [2]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

In [3]:
proj_dir="/master/nplatt/sch_hae_scan"

In [4]:
Path("{}/results/snpeff".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/snpeff".format(proj_dir))

In [ ]:
%%bash 

conda run -n scan_snpeff --cwd . --live-stream \
    java -jar snpEff.jar build -noCheckCds -noCheckProtein -gff3 -v GCF_000699445.3

In [4]:
# %%bash 

# conda run -n scan_snpeff --cwd . --live-stream \
#     java -jar -Xmx64g snpEff.jar \
#         ann \
#         -csvStats target_loci_snpeff.csv \
#         -i vcf \
#         -o vcf \
#         -no-downstream \
#         -no-intergenic \
#         -no-intron \
#         -no-upstream \
#         -canon \
#         -interval ../outlier_table/target_introgressed_loci.bed \
#         -fastaProt target_loci_snpeff.fas \
#         GCF_000699445.3 \
#         ../post_phase_filtering/autosomes_unrelated.vcf \
#         >target_loci_snpeff.vcf

In [ ]:
%%bash 

conda run -n scan_snpeff --cwd . --live-stream \
    java -jar -Xmx64g snpEff.jar \
        ann \
        -csvStats merged_target_loci_snpeff.csv \
        -i vcf \
        -o vcf \
        -no-downstream \
        -no-intergenic \
        -no-intron \
        -no-upstream \
        -canon \
        -interval ../outlier_table/merged_target_loci.bed \
        -fastaProt merged_target_loci.fas \
        GCF_000699445.3 \
        ../outlier_table/merged_target_loci.vcf \
        >merged_target_loci.vcf

In [ ]:
grep -v "#" merged_target_loci.vcf | grep -i -e MISSENSE -e NONSENSE | cut -f1,2 >missense_nonsense.pos
vcftools --vcf merged_target_loci.vcf --positions missense_nonsense.pos --recode --recode-INFO-all --stdout >missense_nonsense.vcf

In [ ]:
cat ../pca/pca_df.csv | cut -f1,37 -d"," | grep ,sb | cut -f1 -d"," > sb.list
cat ../pca/pca_df.csv | cut -f1,37 -d"," | grep ,sh_nw | cut -f1 -d"," > sh_nw.list
cat ../pca/pca_df.csv | cut -f1,37 -d"," | grep ,sh_se | cut -f1 -d"," > sh_se.list

In [ ]:
vcftools --vcf missense_nonsense.vcf --weir-fst-pop sh_nw.list --weir-fst-pop sh_se.list --stdout >nw_vs_se_fst.tsv
vcftools --vcf missense_nonsense.vcf --weir-fst-pop sh_nw.list --weir-fst-pop sb.list --stdout >nw_vs_sb_fst.tsv


In [14]:
se_fst=pd.read_csv("nw_vs_se_fst.tsv", sep="\t", header=0)
sb_fst=pd.read_csv("nw_vs_sb_fst.tsv", sep="\t", header=0)

In [19]:
fst_df=se_fst.copy()
fst_df["sb_fst"] = sb_fst["WEIR_AND_COCKERHAM_FST"]
fst_df.columns=["chrom", "pos", "se_fst", "sb_fst"]
fst_df

,chrom,pos,se_fst,sb_fst
0,NC_067199.1,28479589,-0.001747,-0.011537
1,NC_067199.1,28480384,0.481050,NaN
2,NC_067199.1,28489220,0.984154,-0.006964
3,NC_067199.1,28489259,0.992194,-0.002019
4,NC_067199.1,28492980,NaN,NaN
...,...,...,...,...
984,NC_067200.1,10414986,NaN,NaN
985,NC_067200.1,10415014,NaN,NaN
986,NC_067200.1,10415052,NaN,0.187130
987,NC_067200.1,10415058,NaN,NaN


In [ ]:
vcftools --vcf missense_nonsense.vcf --keep sh_nw.list --freq2 --stdout >nw_freq.tsv
vcftools --vcf missense_nonsense.vcf --keep sh_se.list --freq2 --stdout >se_freq.tsv
vcftools --vcf missense_nonsense.vcf --keep sb.list --freq2 --stdout >sb_freq.tsv
paste nw_freq.tsv se_freq.tsv sb_freq.tsv |cut -f1,2,4,5,6,10,11,12,16,17,18 | sed 1d >freq.raw.tbl



In [30]:
f_df=pd.read_csv("freq.raw.tbl", sep="\t", header=None)
f_df.columns=["chrom", "pos", "nw_n", "nw_r_af", "nw_a_af", "se_n", "se_r_af", "se_a_af", "sb_n", "sb_r_af", "sb_a_af"]
f_df["se_fst"] = fst_df["se_fst"]
f_df["sb_fst"] = fst_df["sb_fst"]
f_df.to_csv("freq.mod.tbl", sep="\t", header=True, index=False)
f_df

,chrom,pos,nw_n,nw_r_af,nw_a_af,se_n,se_r_af,se_a_af,sb_n,sb_r_af,sb_a_af,se_fst,sb_fst
0,NC_067199.1,28479589,130,0.992308,0.007692,104,1.00000,0.00000,40,1.00000,0.00000,-0.001747,-0.011537
1,NC_067199.1,28480384,136,1.000000,0.000000,106,0.54717,0.45283,40,1.00000,0.00000,0.481050,NaN
2,NC_067199.1,28489220,142,0.985915,0.014084,112,0.00000,1.00000,40,1.00000,0.00000,0.984154,-0.006964
3,NC_067199.1,28489259,144,0.993056,0.006944,114,0.00000,1.00000,42,0.97619,0.02381,0.992194,-0.002019
4,NC_067199.1,28492980,126,1.000000,0.000000,104,1.00000,0.00000,40,1.00000,0.00000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,NC_067200.1,10414986,156,1.000000,0.000000,98,1.00000,0.00000,42,1.00000,0.00000,NaN,NaN
985,NC_067200.1,10415014,152,1.000000,0.000000,94,1.00000,0.00000,40,1.00000,0.00000,NaN,NaN
986,NC_067200.1,10415052,138,1.000000,0.000000,104,1.00000,0.00000,40,0.90000,0.10000,NaN,0.187130
987,NC_067200.1,10415058,138,1.000000,0.000000,106,1.00000,0.00000,40,1.00000,0.00000,NaN,NaN


In [32]:
fixed_snvs_df = f_df.loc[abs(f_df["nw_a_af"]-f_df["se_a_af"]) > 0.95].reset_index(drop=True)
fixed_snvs_df.to_csv("fixed_aa_snvs.csv", sep=",", header=True, index=False)
fixed_snvs_df

,chrom,pos,nw_n,nw_r_af,nw_a_af,se_n,se_r_af,se_a_af,sb_n,sb_r_af,sb_a_af,se_fst,sb_fst
0,NC_067199.1,28489220,142,0.985915,0.014084,112,0.000000,1.000000,40,1.000000,0.000000,0.984154,-0.006964
1,NC_067199.1,28489259,144,0.993056,0.006944,114,0.000000,1.000000,42,0.976190,0.023810,0.992194,-0.002019
2,NC_067199.1,28495283,130,0.984615,0.015385,102,0.000000,1.000000,40,1.000000,0.000000,0.982642,-0.006227
3,NC_067199.1,28495360,120,0.983333,0.016667,100,0.000000,1.000000,44,1.000000,0.000000,0.981680,-0.004034
4,NC_067199.1,28498519,138,0.992754,0.007246,78,0.000000,1.000000,44,1.000000,0.000000,0.990673,-0.010391
5,NC_067199.1,28500800,114,1.000000,0.000000,88,0.000000,1.000000,38,1.000000,0.000000,1.000000,NaN
6,NC_067199.1,28506143,134,0.985075,0.014925,110,0.000000,1.000000,34,1.000000,0.000000,0.983496,-0.009076
7,NC_067199.1,28506315,132,0.984848,0.015152,108,0.009259,0.990741,38,1.000000,0.000000,0.974523,-0.007147
8,NC_067199.1,28506438,128,0.984375,0.015625,110,0.000000,1.000000,36,1.000000,0.000000,0.983072,-0.007747
9,NC_067199.1,28516369,136,0.985294,0.014706,100,0.000000,1.000000,40,1.000000,0.000000,0.982945,-0.006616


In [33]:
fixed_snvs_df[["chrom", "pos"]].to_csv("fixed_aa_snvs.pos", sep="\t", header=False, index=False)

In [ ]:
 vcftools --vcf merged_target_loci.vcf --positions fixed_aa_snvs.pos --recode --recode-INFO-all --stdout >fixed_aa_snvs.vcf